In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
import json
import os
import quaternion
import h5py
import models

#import capsulelayers as capslayers
#import capsulenet as capsnet

C:\Python37\lib\site-packages\quaternion\numba_wrapper.py:20: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)
C:\Python37\lib\site-packages\quaternion\calculus.py:475: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from scipy, which means that derivatives
and integrals will use less accurate finite-differencing
techniques.  You may want to install scipy.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)


In [2]:
# Preprocess JSON data and create the dataset
# TODO: set as argument
test_dataset_folder_path     = "C:/Users/BORA/Desktop/lm_test_bop19"
test_split                   = "test"
output_classes = 3 # Can vary depending on the amount of objects

# Save the labels. 
object_ids = []
poses      = []
bboxes     = []

for scene in os.listdir(os.path.join(test_dataset_folder_path, test_split)):
    scene_gt_path      = os.path.join(test_dataset_folder_path, test_split, scene, "scene_gt.json")
    scene_gt_info_path = os.path.join(test_dataset_folder_path, test_split, scene, "scene_gt_info.json")
    
    with open(scene_gt_path, 'r') as scene_gt_file:
        scene_gt_json = json.load(scene_gt_file)
        for img_gt_key in scene_gt_json:
            translation = scene_gt_json[img_gt_key][0]['cam_t_m2c']
            rotation    = scene_gt_json[img_gt_key][0]['cam_R_m2c']
            rotation    = np.reshape(rotation, (3,3))
            rotation    = quaternion.from_rotation_matrix(rotation)
            rotation    = quaternion.as_float_array(rotation)
            obj_id      = scene_gt_json[img_gt_key][0]['obj_id']

            label_id     = []   # One-hot encoding
            label_pose   = []   # Transform + Quaternion

            for index in range(output_classes):
                if index==obj_id-1: # todo: must be -1 later!
                    label_id.append(1.0)
                else:
                    label_id.append(0.0)
            label_id = np.asarray(label_id)
            object_ids.append(label_id)

            label_pose.append(translation[0])
            label_pose.append(translation[1])
            label_pose.append(translation[2])
            label_pose.append(rotation[0])
            label_pose.append(rotation[1])
            label_pose.append(rotation[2])
            label_pose.append(rotation[3])

            label_pose = np.asarray(label_pose)
            poses.append(label_pose)
            

    with open(scene_gt_info_path, 'r') as scene_gt_info_file:
        scene_gt_info_json = json.load(scene_gt_info_file)
        for img_gt_key in scene_gt_info_json:
            labels_bboxes = [] # BBox coords
            bbox = scene_gt_info_json[img_gt_key][0]['bbox_obj']
            labels_bboxes.append(bbox[0])
            labels_bboxes.append(bbox[1])
            labels_bboxes.append(bbox[2])
            labels_bboxes.append(bbox[3])

            labels_bboxes = np.asarray(labels_bboxes)
            bboxes.append(labels_bboxes)


In [3]:
test_images = []
i = 0
for scene in os.listdir(os.path.join(test_dataset_folder_path, test_split)):
    rgb_images_path = os.path.join(test_dataset_folder_path, test_split, scene, "rgb")

    for img in os.listdir(rgb_images_path):
        # Open the image file
        img = tf.io.read_file(os.path.join(rgb_images_path, img))
        # convert the compressed string to a 3D uint8 tensor
        img = tf.image.decode_png(img, channels=3)
        # Use `convert_image_dtype` to convert to floats in the [0,1] range.
        img = tf.image.convert_image_dtype(img, tf.float32)
        # resize the image to the desired size.
        img = tf.image.resize(img, [224, 224])
        # add to the image dataset
        test_images.append(img.numpy())

test_dataset = tf.data.Dataset.from_tensor_slices(({'input': test_images}, {'class_output': object_ids, 'pose_output': poses, 'bbox_output': bboxes}))
test_dataset = test_dataset.batch(1)
print(test_images[0].shape)

(224, 224, 3)


In [4]:
test_dataset

<BatchDataset shapes: ({input: (None, 224, 224, 3)}, {class_output: (None, 3), pose_output: (None, 7), bbox_output: (None, 4)}), types: ({input: tf.float32}, {class_output: tf.float64, pose_output: tf.float64, bbox_output: tf.int32})>

In [30]:
# Prepare for evaluation
model = MobilePoseNet(output_classes)
checkpoint_path = "mobile_pose0035.h5"
model.load_weights(checkpoint_path)
model.summary()
#tf.keras.utils.plot_model(model, 'model.png', show_shapes=True, show_layer_names=True, dpi=600)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss=[tf.keras.losses.CategoricalCrossentropy(), 
                                         tf.keras.losses.MeanSquaredError(), 
                                         tf.keras.losses.MeanSquaredError()], 
                                   metrics={'class_output': [tf.keras.metrics.CategoricalAccuracy()],
                                            'pose_output': [tf.keras.metrics.MeanSquaredError()],
                                            'bbox_output': [tf.keras.metrics.MeanSquaredError()]})

model.evaluate(test_dataset)


Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_81 (Conv2D)              (None, 109, 109, 8)  1184        input[0][0]                      
__________________________________________________________________________________________________
conv2d_82 (Conv2D)              (None, 52, 52, 16)   6288        conv2d_81[0][0]                  
__________________________________________________________________________________________________
dropout_20 (Dropout)            (None, 52, 52, 16)   0           conv2d_82[0][0]                  
____________________________________________________________________________________________

[6406.212383829753,
 0.5832041,
 2747.43,
 3658.198,
 0.33333334,
 2747.43,
 3658.198]

In [0]:
# TFLite Conversion
#converter = tf.lite.TFLiteConverter.from_keras_model(model)
#model = converter.convert()
#open('mobilePoseNet.tflite', 'wb').write(model)